In [16]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron


iris= load_iris()
x = iris.data[:, (2,3)] # petal length, petal width
y = (iris.target == 0).astype(np.int) # iris setosa

In [3]:
per_clf = Perceptron(random_state = 42)
per_clf.fit(x, y)
y_pred = per_clf.predict([[2,0.5], [1,2], [3,4]])

In [4]:
print(y_pred)

[1 0 0]


In [5]:
# using the stochastic gradient descent algorithm
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(loss = 'perceptron', learning_rate = 'constant', eta0 = 1, penalty = None)
sgd.fit(x, y)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=1, fit_intercept=True, l1_ratio=0.15, learning_rate='constant',
       loss='perceptron', n_iter=5, n_jobs=1, penalty=None, power_t=0.5,
       random_state=None, shuffle=True, verbose=0, warm_start=False)

In [6]:
y_pred = sgd.predict([[2,0.5],[1,2],[3,4]])
print(y_pred)

[1 0 0]


In [8]:
import tensorflow as tf
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

import matplotlib.pyplot as plt
image_index = 7777 # You may select anything up to 60,000
print(y_train[image_index]) # The label is 8
plt.imshow(x_train[image_index], cmap='Greys')

8


In [9]:
# Reshaping the array to 4-dims so that it can work with the Keras API
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)
# Making sure that the values are float so that we can get decimal points after division
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# Normalizing the RGB codes by dividing it to the max RGB value.
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print('Number of images in x_train', x_train.shape[0])
print('Number of images in x_test', x_test.shape[0])

x_train shape: (60000, 28, 28, 1)
Number of images in x_train 60000
Number of images in x_test 10000


In [10]:
# Importing the required Keras modules containing model and layers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
# Creating a Sequential Model and adding the layers
model = Sequential()
model.add(Conv2D(28, kernel_size=(3,3), input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten()) # Flattening the 2D arrays for fully connected layers
model.add(Dense(128, activation=tf.nn.relu))
model.add(Dropout(0.2))
model.add(Dense(10,activation=tf.nn.softmax))

Using TensorFlow backend.


In [11]:
# training an MLP with TensorFlow's high-level API on the Mnist image dataset



feature_cols= tf.contrib.learn.infer_real_valued_columns_from_input(x_train)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units = [300,100], n_classes = 10, 
                                        feature_columns = feature_cols)
dnn_clf = tf.contrib.learn.SKCompat(dnn_clf) # if Tensorflow >= 1.1
casted_y_train = tf.cast(y_train, tf.int32)
casted_x_train = tf.cast(x_train, tf.int32)
dnn_clf.fit(x_train.astype(np.int32), y_train.astype(np.int32), batch_size = 50, steps = 40000)


from sklearn.metrics import  accuracy_score
y_pred = dnn_clf.predict(x_test)
accuracy_score(y_test, y_pred['classes'])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000000010E623C8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\ACER\\AppData\\Local\\Temp\\tmpbosr9pnu'}
Instructions for updating:
Please switch to tf.train.get_global_step
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\ACER\AppData\Local\Temp\tmpbosr9pnu\model.ckpt.
INFO:tensorflow:loss = 2.28806, step = 1
INFO:tensorflow:global_step/

INFO:tensorflow:global_step/sec: 80.4826
INFO:tensorflow:loss = 1.26517, step = 7201 (1.233 sec)
INFO:tensorflow:global_step/sec: 84.0334
INFO:tensorflow:loss = 1.301, step = 7301 (1.200 sec)
INFO:tensorflow:global_step/sec: 88.6916
INFO:tensorflow:loss = 1.69072, step = 7401 (1.128 sec)
INFO:tensorflow:global_step/sec: 84.2103
INFO:tensorflow:loss = 1.52784, step = 7501 (1.265 sec)
INFO:tensorflow:global_step/sec: 82.6444
INFO:tensorflow:loss = 1.44984, step = 7601 (1.133 sec)
INFO:tensorflow:global_step/sec: 83.3331
INFO:tensorflow:loss = 1.51374, step = 7701 (1.200 sec)
INFO:tensorflow:global_step/sec: 85.6529
INFO:tensorflow:loss = 1.28027, step = 7801 (1.168 sec)
INFO:tensorflow:global_step/sec: 93.4574
INFO:tensorflow:loss = 1.54528, step = 7901 (1.063 sec)
INFO:tensorflow:global_step/sec: 99.7503
INFO:tensorflow:loss = 1.49403, step = 8001 (1.000 sec)
INFO:tensorflow:global_step/sec: 98.039
INFO:tensorflow:loss = 1.61453, step = 8101 (1.020 sec)
INFO:tensorflow:global_step/sec: 

INFO:tensorflow:loss = 1.44628, step = 15601 (1.208 sec)
INFO:tensorflow:global_step/sec: 99.7491
INFO:tensorflow:loss = 1.33024, step = 15701 (1.010 sec)
INFO:tensorflow:global_step/sec: 80.4817
INFO:tensorflow:loss = 1.64084, step = 15801 (1.235 sec)
INFO:tensorflow:global_step/sec: 88.8875
INFO:tensorflow:loss = 1.35837, step = 15901 (1.125 sec)
INFO:tensorflow:global_step/sec: 90.0887
INFO:tensorflow:loss = 1.42869, step = 16001 (1.113 sec)
INFO:tensorflow:global_step/sec: 90.2921
INFO:tensorflow:loss = 1.39912, step = 16101 (1.110 sec)
INFO:tensorflow:global_step/sec: 93.8953
INFO:tensorflow:loss = 1.45273, step = 16201 (1.058 sec)
INFO:tensorflow:global_step/sec: 93.8953
INFO:tensorflow:loss = 1.31705, step = 16301 (1.068 sec)
INFO:tensorflow:global_step/sec: 92.1645
INFO:tensorflow:loss = 1.68993, step = 16401 (1.090 sec)
INFO:tensorflow:global_step/sec: 65.7885
INFO:tensorflow:loss = 1.63578, step = 16501 (1.528 sec)
INFO:tensorflow:global_step/sec: 60.9747
INFO:tensorflow:loss

INFO:tensorflow:loss = 1.34382, step = 24001 (1.265 sec)
INFO:tensorflow:global_step/sec: 75.4705
INFO:tensorflow:loss = 1.4186, step = 24101 (1.328 sec)
INFO:tensorflow:global_step/sec: 74.7652
INFO:tensorflow:loss = 1.28035, step = 24201 (3.738 sec)
INFO:tensorflow:global_step/sec: 18.3483
INFO:tensorflow:loss = 1.56446, step = 24301 (3.048 sec)
INFO:tensorflow:global_step/sec: 68.1421
INFO:tensorflow:loss = 1.63943, step = 24401 (1.485 sec)
INFO:tensorflow:global_step/sec: 63.4911
INFO:tensorflow:loss = 1.50735, step = 24501 (1.560 sec)
INFO:tensorflow:global_step/sec: 72.071
INFO:tensorflow:loss = 1.56428, step = 24601 (1.383 sec)
INFO:tensorflow:global_step/sec: 70.4215
INFO:tensorflow:loss = 1.36938, step = 24701 (1.418 sec)
INFO:tensorflow:global_step/sec: 73.6637
INFO:tensorflow:loss = 1.37617, step = 24801 (1.355 sec)
INFO:tensorflow:global_step/sec: 69.5642
INFO:tensorflow:loss = 1.30385, step = 24901 (1.445 sec)
INFO:tensorflow:global_step/sec: 71.4275
INFO:tensorflow:loss =

INFO:tensorflow:loss = 1.4445, step = 32401 (2.183 sec)
INFO:tensorflow:global_step/sec: 47.5052
INFO:tensorflow:loss = 1.51248, step = 32501 (2.085 sec)
INFO:tensorflow:global_step/sec: 60.7893
INFO:tensorflow:loss = 1.58751, step = 32601 (1.648 sec)
INFO:tensorflow:global_step/sec: 69.807
INFO:tensorflow:loss = 1.40923, step = 32701 (1.428 sec)
INFO:tensorflow:global_step/sec: 55.1716
INFO:tensorflow:loss = 1.15958, step = 32801 (1.813 sec)
INFO:tensorflow:global_step/sec: 77.3683
INFO:tensorflow:loss = 1.31919, step = 32901 (1.290 sec)
INFO:tensorflow:global_step/sec: 62.5968
INFO:tensorflow:loss = 1.65861, step = 33001 (1.603 sec)
INFO:tensorflow:global_step/sec: 62.499
INFO:tensorflow:loss = 1.56638, step = 33101 (1.605 sec)
INFO:tensorflow:global_step/sec: 62.4015
INFO:tensorflow:loss = 1.32023, step = 33201 (1.595 sec)
INFO:tensorflow:global_step/sec: 56.3372
INFO:tensorflow:loss = 1.41714, step = 33301 (1.775 sec)
INFO:tensorflow:global_step/sec: 75.4705
INFO:tensorflow:loss = 

0.91449999999999998

In [12]:
# training a DNN using plain tensorflow
import tensorflow as tf
import numpy as np
n_inputs = 28 *28 # MNIST no features is 28 by 28(one feature per pixel)
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

# since we do not know how many instances each training batch will contain, so the shape
# of X is (None, n_inputs) and y is (None)

X = tf.placeholder(tf.float32, shape = (None, n_inputs), name = 'X')
y = tf.placeholder(tf.int64, shape = (None), name= 'y')

# creating the two hidden layers and the output, the output layer uses a softmax activation
# function instead of a RELU activation function.
# we will create a nuuron_layer() function that we will use to create one layer at a time
# It will need parameters to specify the inputs, the number of neutrons, the activation function,
# and the name of the layer....

def neuron_layer(X, n_neurons, name, activation = None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs + n_neurons)
        init = tf.truncated_normal((n_inputs, n_neurons),stddev = stddev)
        W = tf.Variable(init,name = 'kernel')
        b = tf.Variable(tf.zeros(n_neurons), name = 'bias')
        z = tf.matmul(X, W) + b
        if activation is not None: # that is if activation parameter is provided
            # such as tf.nn.relu(i.e, max(0,z)), then the code returns activation(Z), or
            #else it just returns Z.
            return activation(z)
        else:
            return z
        
        
        
with tf.name_scope('dnn'):
    hidden1 = neuron_layer(X, n_hidden1, name = 'hidden1', activation = tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name = 'hidden2', activation = tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name = 'outputs')
    
    
# using tensorflow built in support for building layers
#with tf.name_scope('dnn'):
#    hidden1 = tf.layers.dense(X, n_hidden1, name = 'hidden1', activation = tf.nn.relu)
#    hidden2 = tf.layers.dense(x, n_hidden2, name = 'hidden2', activation = tf.nn.relu)
#    logits = tf.layers.dense(hidden2, n_outputs, name = 'outputs')
    
    
# applyiing cross Entropy, cross entropy will penalize models that estimates a low
# probability for the target class. using sparse_softmax_cross_entropy_with_logits()
# it computes the cross entropy based on the 'logits' (i.e., the output of the network)
# before going through the softmax activation function), and it expects labels in the form
# of integers ranging from 0 to the number of classes minus 1(in our case, from 0 to 9). This
# will give us a 1D tensor containing the cross entropy for each instance. 
# we can then use Tensorflow's reduce_mean() function to compute the mean cross entropy over all
# instances.

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits)
    loss = tf.reduce_mean(xentropy, name = 'loss')


In [13]:
# now getting the GradientDescentOptimizer that will tweak the model parameters, to
# minimize the cost function

learning_rate = 0.01

with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    trainin_op = optimizer.minimize(loss)
    
    
# now to specify how to evaluate our model, simply using accuracy as our performance measure
# to determine if the neural network's prediction is correct by checking whether or not
# the highest logit corresponds to the target class... using the in_to_k() function.
# this returns a 1D tensor full of boolean values, so we need to cast these booleans to floats
# and the compute the average.

with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y,1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

# Execution phase

In [ ]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
n_epochs = 40
batch_size = 50



In [ ]:
with tf.Session() as ess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples//batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict = {X:X_batch, y:y_batch})
            acc_train = accuracy.eval(feed_dict = {X:X_batch, y:y_batch})
            acc_val = accuracy.eval(feed_dict = {X:mnist.validation.images, y:mnist.validation.labels})
            print(epoch, 'Train accuracy:', acc_train, 'Val accuracy:', acc_val)
            
        save_path = saver.save(sess, './my_model_final.ckpt')

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, './my_model_final.ckpt')
    X_new_scaled = [....] # some new images(scaled from 0 to 1)
    z = logits.eval(feed_dict = {X:X_new_scaled})
    y_pred = np.argmax(z, axis = 1)

In [ ]:
# fine-tuned neural network hyperparameters



# for the output layer, the softmax activation function is generally a good choice
# for classification tasks when the classes are mutually exclusive. When they are not
# mutally exclusive(or when they are just two classes), you generally want to use the 
# logistic function. For regression tasks, you can simply use no activation function at all
# for the output layer.